Firstly, we need to upload the raw battery xlsx spreadsheet. Then we read it into a dataframe (b). Then plot out by hubid.

In [ ]:
#!ls
#!pwd
try:
    from google.colab import files
    uploaded = files.upload()
except:
    pass # failed to load module on laptop

In [ ]:
import pandas as pd
import dateutil.parser as du
from datetime import datetime as dt
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()
csv = True

try:
    if csv:
        full_set = pd.read_csv ('batteryDetails.csv', parse_dates=[5])
        full_set.columns = ['Hub Mac', 'ClientID', 'Device Slot ID + Node', 'Device Type', 'Epoch TS', 'Date Time',
       'Battery']
    else:
        full_set = pd.read_excel ('battery analysis.xlsx')
except FileNotFoundError as fnf:
    print ("cannot find excel file with battery readings:", fnf)

full_set ['ClientID'] = full_set ['ClientID'].apply (lambda x: x.strip())

print (full_set['Date Time'][0].__class__)

Remove data from before the last HubV2 update as the quality is unknown before then.

In [ ]:
valid_set= full_set[full_set ['Date Time'] > du.parse ("2019-5-15")]

print (min (valid_set['Date Time']))

print (valid_set [valid_set ['ClientID'].isin (bad_hubs)])

Remove known bad hubs from this set of data: D, E, F and G. The hubids can be identified in the section below [Identifying Devices](#Identifying-Devices)

In [ ]:
bad_hubs = ['5aa147d1bbda2e2cc46bb73e', '5a992264bbda2e2cc46b2f21', '5a992175bbda2e2cc46b2d23', '5b1e3f62c7fe056c961f0639']
print ("starting number of devices", len (valid_set))
valid_set = valid_set [~valid_set ['ClientID'].isin (bad_hubs)]
print ("number of devices after ignoring {} hubs: {}".format (len (bad_hubs), len (valid_set)))

C's hub changed at the end of July. Therefore are all of her devices before that day should be ignored.

A installed on 31/7, B on 3/9. Trim these times out.

In [ ]:
# code to strip out the bad data from more recently installed hubs

late_installs = {'5a8eef6b0446f007bd4eff0d': du.parse ("2019-7-31"), '5d3ea83ac7fe0554e22ae04c': du.parse ("2019-7-31"),
                'undefined': du.parse ("2019-9-3")}
#print (late_installs)

print ("before removing pre-install data {}".format (len (valid_set)))

#print (valid_set [valid_set ['ClientID'] == '5a8eef6b0446f007bd4eff0d']["Date Time"])

#print (valid_set.__class__)

for k, v in late_installs.items ():
    valid_set.drop (valid_set [(valid_set.ClientID == k) & (valid_set['Date Time'] <= v)].index, inplace=True)
    #print (valid_set [(valid_set.ClientID == k) ])
    

print ("after removing pre-install data {}".format (len (valid_set)))

Plot out the battery reporting for each device, by hub. The purpose of the plot is to highlight gross errors / inconsistencies.

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline

# plot out the battery events into existing matplotib azes. Annoyingly, matplotlib creates differently
# shaped sets of axes for 1 column vs >1 column.
def plotting (axs, figs, rows, cols):
    count=0
    r,c = 0, 0
    fig = 0
    hubs = []
    axi = axs[fig]
#fig, ax = plt.subplots (nrows=8, ncols=4, sharex=True , figsize = (18,32))
    #fig, ax = plt.subplots (nrows=8, ncols=4, sharex=True , figsize = (20, 12))

#for h, d in valid_set.groupby ('Hub Mac'):
    for h, d in valid_set.groupby ('ClientID'):
  #if '5d25' not in h:
  #  continue
        if cols==1:  # non-orthogonal api :-(    
            ax = axi[r]
        else:
            ax = axi[r,c]
        ax.set_title ("Hub: {}".format (h))
        #print (h)
        h_str = "{}\n".format (h)
        #hubs.append (h)
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        loc=mdates.DayLocator(interval=56)
        timfmt = mdates.AutoDateFormatter(loc)
        ax.xaxis.set_major_formatter (timfmt)
        ax.xaxis.set_major_locator(loc)
        ax.xaxis.set_tick_params(rotation=30)


        for v, g in d.groupby ('Device Slot ID + Node'):
#    ax.scatter (x=g['Date Time'], y=g['Battery'], alpha=0.3, label=v[0:5])
            ax.plot_date (x=g['Date Time'], y=g['Battery'], alpha=0.3, label=v) #[0:8])
            #print (v)
            h_str += "{}\n".format (v)
# Put a legend to the right of the current axis
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#    ax.legend()

#  ax[count].xaxis.grid(True)
        r += 1
        if r >= rows:
            c += 1
            if c >= cols:
                c = 0
                fig += 1
                if fig < figs:
                    ax = axs[fig]
            r = 0
        hubs.append (h_str)
    return hubs

figs = 1
# this will work with >1 column, but the shape needs to be rectangular.
rows, cols = len (set (valid_set ["ClientID"])), 1
ax=[]

# create the layout of the axes for plotting
for i in range (figs):
    fig, axes = plt.subplots (nrows=rows, ncols=cols, sharex=True, sharey=True, figsize = (20, 35))
    ax.append (axes)

# plot the results and grab the identity of the hubs/devices from the spreadsheet
hubs = plotting (ax, figs, rows, cols)

# do not need to save these now. Previously, this was an intermediate store
#with open ('hubs', 'w') as of:
#    for h in hubs:
#        of.write ("{}\n".format (h))

In [ ]:
min (valid_set['Date Time'])

In [ ]:
ls=valid_set

How many battery powered devices are there with the valid dataset.

In [ ]:
# enumerate relevant devices and count them. The list comprehension enumerates the earliest battery time for each device
len ([(x, min (ls[ls['Device Slot ID + Node']==x]['Battery'])) for x in set (ls ['Device Slot ID + Node'])])

How many of the devices have ever had a battery level lower than 100? Some of these have always been < 100

In [ ]:
low = [(x, min (ls[ls['Device Slot ID + Node']==x]['Battery'])) for x in set (ls ['Device Slot ID + Node']) if min (ls[ls['Device Slot ID + Node']==x]['Battery']) < 100]
print (len (low))

# Devices that Have Reported loss of Battery Capacity
Print out the device slot/ZWave node id, and all the readings from the devices that have ever had a battery reading < 100. This plot shows that at least one device had non-monotonic readings. We believe, but cannot prove, that it has had its battery replaced. So we will remove it later in the process.

In [ ]:
for did in [x[0] for x in low]:
    print (did)
    print (valid_set[valid_set['Device Slot ID + Node'] == did][['Battery', 'Date Time']])

# Identifying Devices
The following tests which hubs/devices identified in the excel spreadsheet are in the production environment and reports where there are discrepencies. The discrepencies require further investigation and either elimination from the analysis set with specific filters or the add to the information available.

For each hub, the owner is identified, and the set of devices that are relevant. The devices from the spreadsheet relate to device slots which contain physical devices (possibly >1 in each slot, eg for door sensors). The code prints out the types of each device, but only includes the slot information if the underlying physical device is battery powered.

In [ ]:
import subprocess as sb
import json

# originally, these values were saved from above and picked up here. Now all in one program
# hubs = open ('hubs').readlines ()


# this procedure pulls down from pumphouse the device identifiers for devices currently connected to hubs with a
# given hubid. The hubid comes from the spreadsheet.
def devs_for_hub_from_live (h):
    #for h in hubs: #["5a8eef6b0446f007bd4eff0d"]: #hubs:
    devs = []
    h = h.strip ()
    #print ("hubid", h)
    js = sb.check_output ('curl --header "Content-Type: application/json" https://homeinstead.anthropos.io/PumpHouse/rest/v1/iot/users/{}/residences'.format (h), shell=True)
    #print ("js from curl", js)
    cust = json.loads (js)

    print ("\nhub + owner", h, cust['residence'][0]['fn'])
    #print (cust['residence'][0]['devices'])
    for l in cust['residence'][0]['locations']:
        #print (l.keys())
        for slot in l['deviceSlots']:
            for d in slot['devices']:
                print ("physical", d['uuid'], d['model'])
                #devs.append (d['uuid'])
            if all (('power' not in slot ['deviceType'], 'pressure' not in slot ['deviceType'])):   # ignore power meters and BCGs for battery stuff
                print (slot['uuid'], slot ['deviceType'])
                devs.append (slot['uuid'])
    return cust ['residence'][0]['fn'], devs


# minimal bit of OO ;-)
class hub ():
    def __init__ (self, str):
        vals = str.split ()
        self.id = vals [0]
        self.devs = [x.split ('_')[0] for x in vals [1:]]
        #print (self.devs)

from_live = {}
names = {}

from_excel = {}
for h_str in hubs:
    h = h_str.split ()[0]
    #print ("hh", h)
    from_excel [h] = hub (h_str)
    names [h.strip ()], from_live [h.strip()] = devs_for_hub_from_live (h.strip ())

# check that the hubs on pumphouse and in the excel report are consistent
assert from_excel.keys() == from_live.keys()

print ("\n\nBreakdown of devices reporting battery levels vs devices currently in production")
for k in from_excel.keys ():
    print ()
    print (names[k].strip (), k)
    #print (from_excel [k].devs, from_live[k])
    print ("count of devices currently in use and that have ever been in use", len (set (from_excel[k].devs).intersection (set (from_live[k]))))
    print ("historical, not current", len (set (from_excel [k].devs).difference (set (from_live[k]))))
    print ("current not historical", len (set (from_live [k]).difference (set (from_excel[k].devs))))




# Next Steps

1. Use the data from [#Identifying Devices](#Identifying-Devices) and [#Devices that Have Reported loss of Battery Capacity](#Devices-that-Have-Reported-loss-of-Battery-Capacity) to categorise the devices by type.
1. Evaluate the overall failure rate per device month of use
1. Estimate the rate of drop and the variation of per device type from battery first reporting a drop in capacity. (e.g. "there's a roughly linear capacity drop of 10% per week for Door Sensors, with a 95% CI of 5 and 25)

If it looks like too much of a challenge to code up the estimation, then I can work those through and share what the code looks like.

In [ ]:
print (from_excel.keys())
#print ([(x.devs, x.id) for x in from_excel ['5d25b901c7fe05081ab12ce2']])
#print (from_excel ['5d25b901c7fe05081ab12ce2'].devs)